<a href="https://colab.research.google.com/github/ricardoskewes/68610finalprojectgroup48/blob/colab_branch/supertruly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
rm -rf 68610finalprojectgroup48
git clone https://github.com/ricardoskewes/68610finalprojectgroup48.git

Cloning into '68610finalprojectgroup48'...


In [2]:
!pip install sacrebleu simpletransformers
!pip install wordfreq
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0

In [3]:

# Append the cloned repository to sys.path

import sys
sys.path.append("/content/68610finalprojectgroup48")
projectDir = "/content/68610finalprojectgroup48/tweepfake"
sys.path.insert(0, projectDir)
resultsDir = projectDir+"/data/results"
import datetime
import math
import numpy as np
import pandas as pd
import torch
import wandb
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
from wordfreq import word_frequency
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from transformers import (BertForSequenceClassification, BertTokenizerFast, Trainer,
                          TrainingArguments, EarlyStoppingCallback, DataCollatorWithPadding)
from datasets import Dataset
import random
# from simpletransformers.classification import ClassificationModel, ClassificationArgs

# wandb library
import wandb
wandb.login()
WANDB_PROJECT_NAME = "6-861-finalproj"


random_state = 523
torch.manual_seed(random_state)
np.random.seed(random_state)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:

#DataHandler
from DataHandler import DataHandler
dh = DataHandler()

from datasets import Dataset


In [5]:
# Paths to datasets
csvTrainDataset = f"{projectDir}/data/splits/train.csv"
csvValDataset = f"{projectDir}/data/splits/validation.csv"
csvTestDataset = f"{projectDir}/data/splits/test.csv"

# Load datasets
dfTrain = dh.readCSVData(csvTrainDataset)
dfVal = dh.readCSVData(csvValDataset)
dfTest = dh.readCSVData(csvTestDataset)

# Select relevant columns
dfTrainDataset = dfTrain[["screen_name", "text", "account.type"]]
dfValDataset = dfVal[["screen_name", "text", "account.type"]]
dfTestDataset = dfTest[["screen_name", "text", "account.type"]]

# Prepare training data
X_train_all = dfTrainDataset.drop(columns=['screen_name'])
X_train_all.columns = ["text", "label"]

X_val_all = dfValDataset.drop(columns=['screen_name'])
X_val_all.columns = ["text", "label"]

X_test_all = dfTestDataset.drop(columns=['screen_name'])
X_test_all.columns = ["text", "label"]

# Map labels to integers
dictLabels = {"human": 0, "bot": 1}
X_train_all["label"] = X_train_all["label"].map(dictLabels)
X_val_all["label"] = X_val_all["label"].map(dictLabels)
X_test_all["label"] = X_test_all["label"].map(dictLabels)

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(X_train_all, preserve_index=False)
val_dataset = Dataset.from_pandas(X_val_all, preserve_index=False)
test_dataset = Dataset.from_pandas(X_test_all, preserve_index=False)

# Verify the conversion
print(type(train_dataset))  # Should output: <class 'datasets.arrow_dataset.Dataset'>
print(type(val_dataset))    # Should output: <class 'datasets.arrow_dataset.Dataset'>
print(type(test_dataset))   # Should output: <class 'datasets.arrow_dataset.Dataset'>

# -----------------------
# Feature Functions
# -----------------------

spacy_nlp = spacy.load("en_core_web_sm")

def get_spacy_doc(text):
    return spacy_nlp(text)

def switch_spacy_to_text(func):
    def inner1(*args, **kwargs):
        spacy_doc = get_spacy_doc(args[0])
        return func(spacy_doc, **kwargs)
    return inner1

def count_words(text):
    return len(text.split())

def avg_word_length(text):
    words = text.split()
    return np.mean([len(w) for w in words]) if len(words) > 0 else 0

@switch_spacy_to_text
def get_ANP(spacy_doc):
    filtered_tokens = [token for token in spacy_doc if not token.is_punct and not token.is_space]
    total_tokens = len(filtered_tokens)

    adjectives = [token.text for token in filtered_tokens if token.pos_ == "ADJ"]
    nouns = [token.text for token in filtered_tokens if token.pos_ == "NOUN"]
    pronouns = [token.text for token in filtered_tokens if token.pos_ == "PRON"]

    if total_tokens == 0:
        adj_density = 0
        noun_density = 0
        pronoun_density = 0
    else:
        adj_density = len(adjectives) / total_tokens
        noun_density = len(nouns) / total_tokens
        pronoun_density = len(pronouns) / total_tokens
    return adjectives, nouns, pronouns, adj_density, noun_density, pronoun_density

def get_ANP_clean(text):
    _, _, _, adj_density, noun_density, pronoun_density = get_ANP(text)
    return adj_density, noun_density, pronoun_density

@switch_spacy_to_text
def get_capitalizations(spacy_doc):
    capitalizations = [token.text for token in spacy_doc if token.text.isupper()]
    return capitalizations, len(capitalizations) / len(spacy_doc) if len(spacy_doc) > 0 else 0

def get_capitalizations_clean(text):
    _, cap_ratio = get_capitalizations(text)
    return np.round(cap_ratio, 2)

@switch_spacy_to_text
def get_sentiment(spacy_doc):
    polarity, subjectivity = TextBlob(spacy_doc.text).sentiment
    return np.round(polarity, 3), np.round(subjectivity, 3)

@switch_spacy_to_text
def calculate_rarity_scores(spacy_doc, lang='en'):
    adj_rarity_scores = []
    noun_rarity_scores = []
    for token in spacy_doc:
        if token.pos_ == "ADJ":
            freq = word_frequency(token.text.lower(), lang)
            adj_rarity_scores.append(-math.log(freq) if freq > 0 else 0)
        if token.pos_ == "NOUN":
            freq = word_frequency(token.text.lower(), lang)
            noun_rarity_scores.append(-math.log(freq) if freq > 0 else 0)
    res = [0, 0]
    if noun_rarity_scores:
        res[0] = np.median(noun_rarity_scores)
    if adj_rarity_scores:
        res[1] = np.median(adj_rarity_scores)
    return tuple(np.round(res, 3))

@switch_spacy_to_text
def get_punctuation(spacy_doc):
    punctuation = [token.text for token in spacy_doc if token.pos_ == "PUNCT"]
    return punctuation, len(punctuation)/len(spacy_doc) if len(spacy_doc)>0 else 0

def get_punctuation_clean(text):
    _, punct_ratio = get_punctuation(text)
    return np.round(punct_ratio, 2)

# Dictionary of classical features
C = {
    1: count_words,                  # C1: Number of words in sentence
    2: avg_word_length,              # C2: Average word length in sentence
    3: get_ANP_clean,                # C3: Density scores of adjectives, pronouns, and nouns
    4: get_capitalizations_clean,     # C4: Density of capital letters
    5: get_sentiment,                # C5: Sentiment analysis
    6: calculate_rarity_scores,      # C6: Noun and adjective rarity scores
    7: get_punctuation_clean         # C7: Density of punctuation
}

# Model descriptions
model_descriptions = {
    1: 'BERT*(tweet)',
    2: 'BERT*({C1 <SEP> C2 <SEP> ... <SEP> tweet})',
    3: 'BERT*({C1 <FEAT> C2 <FEAT> ... <FEAT> tweet})',
}

# Model-specific arguments
model_args_dict = {
    1: {'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output_dir': 'type1'},
    2: {'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output_dir': 'type2'},
    3: {'separator_token_name': 'FEAT', 'transformer_type': 'bert', 'transformer_name': 'bert-base-cased', 'output_dir': 'type3'},
}

# Hyperparameters for each model
model_hyperparameters = {
    1: {"learning_rate": 3e-5, "train_batch_size": 16, "num_train_epochs": 3},
    2: {"learning_rate": 3e-5, "train_batch_size": 16, "num_train_epochs": 3, "C_ids": [1,2, 3, 4, 5, 6, 7]},
    3: {"learning_rate": 3e-5, "train_batch_size": 16, "num_train_epochs": 3, "C_ids": [1,2, 3, 4, 5, 6, 7]},
}

# -----------------------
# Special Tokens and Encoding Functions
# -----------------------

def create_special_tokens_dict(model_id, model_args_dict):
    additional_tokens = []
    if model_id == 3:
        additional_tokens.append(model_args_dict[model_id]['separator_token_name'])
    return {'additional_special_tokens': additional_tokens}

def initialize_model_with_special_tokens(model_id, model_args_dict, hyperparameters, special_tokens_dict, model_descriptions):
    transformer_name = model_args_dict[model_id]['transformer_name']
    tokenizer = BertTokenizerFast.from_pretrained(transformer_name)
    model = BertForSequenceClassification.from_pretrained(transformer_name, num_labels=2)

    if special_tokens_dict and len(special_tokens_dict['additional_special_tokens']) > 0:
        tokenizer.add_special_tokens(special_tokens_dict)
        model.resize_token_embeddings(len(tokenizer))

    return model, tokenizer

def encode_with_features(example, tokenizer, model_id, model_args_dict, model_hyperparameters):
    """
    Encodes features and tweet, inserting FEAT or SEP tokens between them based on model_id.
    """
    tweet = example['text']

    if model_id == 3:
        separator_token = 'FEAT'
    elif model_id == 2:
        separator_token = tokenizer.sep_token
    else:
        separator_token = tokenizer.sep_token  # Default for model_id=1

    # Get separator token ID
    if model_id == 3:
        separator_token_id = tokenizer.convert_tokens_to_ids(separator_token)
    else:
        separator_token_id = tokenizer.sep_token_id

    # Get list of feature texts based on C_ids
    if 'C_ids' in model_hyperparameters[model_id]:
        C_functions = [C[i] for i in model_hyperparameters[model_id]['C_ids']]
    elif 'C_ids' in model_args_dict[model_id]:
        C_functions = [C[i] for i in model_args_dict[model_id]['C_ids']]
    else:
        C_functions = []

    # Initialize input_ids with [CLS] token
    cls_token_id = tokenizer.cls_token_id
    input_ids = [cls_token_id]

    # Encode each feature and append with [SEP] + separator_token_id
    for i, func in enumerate(C_functions):
        feature_text = func(example['text'])
        if isinstance(feature_text, tuple):
            feature_text = ' '.join(map(str, feature_text))
        else:
            feature_text = str(feature_text)

        # Encode feature text with add_special_tokens=False
        encoded_feature = tokenizer.encode(
            feature_text,
            add_special_tokens=False,
            truncation=True,
            max_length= int(250 / len(C_functions))
        )
        input_ids += encoded_feature

        # Append [SEP]
        input_ids += [tokenizer.sep_token_id]

        # Append separator token (FEAT or SEP) if not the last feature
        if i < len(C_functions) - 1:
            input_ids += [separator_token_id]

    # Insert separator token before tweet
    if len(C_functions) > 0:
        input_ids += [separator_token_id]

    # Encode tweet
    encoded_tweet = tokenizer.encode(
        tweet,
        add_special_tokens=True, #TODO: change if doesnt work
        truncation=True,
        max_length=tokenizer.model_max_length - len(input_ids) - 1  # Reserve for final [SEP]
    )

    input_ids += encoded_tweet[1:] #TODO: change if doesnt work. this gets rid of furst token

    # Append [SEP] at the end
    input_ids += [tokenizer.sep_token_id]

    # Create attention mask
    attention_mask = [1] * len(input_ids)

    # Pad to max_length
    padding_length = tokenizer.model_max_length - len(input_ids)
    if padding_length > 0:
        input_ids += [tokenizer.pad_token_id] * padding_length
        attention_mask += [0] * padding_length

    # Truncate if necessary
    if len(input_ids) > tokenizer.model_max_length:
        input_ids = input_ids[:tokenizer.model_max_length]
        attention_mask = attention_mask[:tokenizer.model_max_length]

    # Assign to example
    example['input_ids'] = input_ids
    example['attention_mask'] = attention_mask
    # Rename 'label' to 'labels' to match model expectations
    example['labels'] = example['label']
    del example['label']

    return example

def preprocess_function(example, tokenizer, model_id, model_args_dict, model_hyperparameters):
    return encode_with_features(example, tokenizer, model_id, model_args_dict, model_hyperparameters)

def prepare_datasets(train_dataset, val_dataset, test_dataset, tokenizer, model_id, model_args_dict, model_hyperparameters):
    # Apply the preprocessing function to all datasets
    train_dataset = train_dataset.map(
        lambda x: preprocess_function(x, tokenizer, model_id, model_args_dict, model_hyperparameters),
        batched=False
    )
    val_dataset = val_dataset.map(
        lambda x: preprocess_function(x, tokenizer, model_id, model_args_dict, model_hyperparameters),
        batched=False
    )
    test_dataset = test_dataset.map(
        lambda x: preprocess_function(x, tokenizer, model_id, model_args_dict, model_hyperparameters),
        batched=False
    )

    # Set format for PyTorch
    train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    return train_dataset, val_dataset, test_dataset

# -----------------------
# Define Metrics for Evaluation
# -----------------------

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, predictions)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

# -----------------------
# Training and Evaluation Functions
# -----------------------

def train_model(train_dataset, val_dataset, model_id, model_hyperparameters, model_args_dict, wandb_project_name, sweeping = True):
    # Initialize wandb run
    current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    run_name = f"Model_{model_id}_{current_time}"
    tags = [
        f"model_{model_id}",
        f"date_{current_time.split('_')[0]}",
    ]
    config_stuff = {**model_args_dict[model_id], **model_hyperparameters[model_id]}
    for param in config_stuff:
        tags.append(f"{param}_{config_stuff[param]}")

    if not sweeping:
      run = wandb.init(
          project=wandb_project_name,
          name=run_name,
          notes=model_descriptions[model_id],
          tags=tags,
          config=config_stuff
      )
    else:
      wandb.log(config_stuff)

    # Initialize model and tokenizer
    special_tokens_dict = create_special_tokens_dict(model_id, model_args_dict)
    model, tokenizer = initialize_model_with_special_tokens(model_id, model_args_dict, model_hyperparameters, special_tokens_dict, model_descriptions)

    # Preprocess datasets
    train_dataset_processed, val_dataset_processed, _ = prepare_datasets(
        train_dataset, val_dataset, test_dataset, tokenizer, model_id, model_args_dict, model_hyperparameters
    )

    # Data collator (handles dynamic padding)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=model_args_dict[model_id]['output_dir'],
        overwrite_output_dir=True,
        num_train_epochs=model_hyperparameters[model_id]['num_train_epochs'],
        per_device_train_batch_size=model_hyperparameters[model_id]['train_batch_size'],
        per_device_eval_batch_size=model_hyperparameters[model_id]['train_batch_size'],
        evaluation_strategy="steps",
        save_steps=100,
        eval_steps=100,
        save_total_limit=1,
        logging_steps=50,
        learning_rate=model_hyperparameters[model_id]['learning_rate'],
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        report_to=["wandb"],
        remove_unused_columns=False  # Critical for custom datasets
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_processed,
        eval_dataset=val_dataset_processed,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Watch the model with wandb
    wandb.watch(model, log='all')

    # Train the model
    trainer.train()

    # Evaluate the model on validation data
    eval_results = trainer.evaluate()
    wandb.log(eval_results)

    # Log an example input
    example = train_dataset_processed[0]['input_ids'].tolist()
    wandb.log({'example_input_ids': example})

    # Save artifact

    artifact = wandb.Artifact(
        name = f'Type_{model_id}',
        type='model',
        description=model_descriptions[model_id],
        metadata={
            'model_id': model_id,
            'model_hyperparams': model_hyperparameters[model_id],
            'model_args': model_args_dict[model_id],
            'random_state': random_state,
        }
    )
    artifact.add_dir(model_args_dict[model_id]['output_dir'])
    wandb.log_artifact(artifact)

    # Finish wandb run
    if not sweeping:
      run.finish()

    return model, tokenizer

def evaluate_model(test_dataset, model_id, model, tokenizer, model_hyperparameters, model_args_dict, wandb_project_name):
    # Preprocess test dataset
    _, _, test_dataset_processed = prepare_datasets(
        train_dataset, val_dataset, test_dataset, tokenizer, model_id, model_args_dict, model_hyperparameters
    )

    # Data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # Define evaluation arguments
    eval_args = TrainingArguments(
        output_dir=model_args_dict[model_id]['output_dir'],
        per_device_eval_batch_size=model_hyperparameters[model_id]['train_batch_size'],
        report_to=["wandb"],
        logging_steps=50
    )

    # Initialize Trainer for evaluation
    trainer = Trainer(
        model=model,
        args=eval_args,
        eval_dataset=test_dataset_processed,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    # Evaluate the model
    eval_results = trainer.evaluate()
    wandb.log(eval_results)

    # Get predictions
    predictions = trainer.predict(test_dataset_processed)
    preds = np.argmax(predictions.predictions, axis=1)
    true_labels = predictions.label_ids

    # Plot confusion matrix
    cm = confusion_matrix(true_labels, preds)
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    wandb.log({"confusion_matrix": wandb.Image(plt)})
    plt.close()

# -----------------------
# Sweep Configuration and Execution (Optional)
# -----------------------

def sweep_train():
    run = wandb.init()
    config = wandb.config
    chosen_model_id = config.get('model_id', 1)

    # Update hyperparameters based on config
    if 'learning_rate' in config:
        model_hyperparameters[chosen_model_id]['learning_rate'] = config.learning_rate
    if 'train_batch_size' in config:
        model_hyperparameters[chosen_model_id]['train_batch_size'] = config.train_batch_size
    if 'num_train_epochs' in config:
        model_hyperparameters[chosen_model_id]['num_train_epochs'] = config.num_train_epochs
    if 'C_ids' in config:
        model_hyperparameters[chosen_model_id]['C_ids'] = config.C_ids

    # Train the model
    model, tokenizer = train_model(
        train_dataset,
        val_dataset,
        chosen_model_id,
        model_hyperparameters,
        model_args_dict,
        WANDB_PROJECT_NAME
    )

    # Evaluate the model
    evaluate_model(
        test_dataset,
        chosen_model_id,
        model,
        tokenizer,
        model_hyperparameters,
        model_args_dict,
        WANDB_PROJECT_NAME
    )

    # Finish the run once everything is done
    #wandb.finish()

sweep_config = {
    'method': 'grid',
    'metric': {'name': 'f1', 'goal': 'maximize'},
    'parameters': {
        'model_id': {
            'values': [2, 3]
        },
        'learning_rate': {
            'values': [5e-5]
        },
        'train_batch_size': {
            'values': [32]
        },
        'C_ids': {
            'values': [[1,2,3,4,5,6,7], [1,2, 3] ]
        }
    }
}


<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [6]:
sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT_NAME)
wandb.agent(sweep_id, sweep_train, count=4)  # Total runs = 4


Create sweep with ID: 51ap3mue
Sweep URL: https://wandb.ai/rskewes-harvard-university/6-861-finalproj/sweeps/51ap3mue


wandb: Agent Starting Run: obt5mehd with config:
wandb: 	C_ids: [1, 2, 3, 4, 5, 6, 7]
wandb: 	learning_rate: 5e-05
wandb: 	model_id: 2
wandb: 	train_batch_size: 32
wandb: Currently logged in as: rskewes (rskewes-harvard-university). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Map:   0%|          | 0/2558 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-24148b249e1b>:371: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.400600,0.359796,0.822763,0.817406,0.831597,0.824441
200,0.345200,0.309726,0.846655,0.877243,0.806424,0.840344
300,0.276200,0.317896,0.846655,0.824533,0.881076,0.851867
400,0.316600,0.286554,0.867072,0.864655,0.870660,0.867647
500,0.284700,0.310636,0.853171,0.852686,0.854167,0.853426
600,0.259500,0.272882,0.862728,0.794366,0.979167,0.877138
700,0.153600,0.283229,0.885752,0.901536,0.866319,0.883577
800,0.204300,0.287646,0.874891,0.896330,0.848090,0.871543
900,0.192700,0.281566,0.896612,0.879568,0.919271,0.898981
1000,0.149900,0.292976,0.896177,0.877585,0.921007,0.898772


epoch,▁
eval/accuracy,▁▃▃▅▄▅▇▆██▆██
eval/f1,▁▂▄▅▄▆▇▅██▆██
eval/loss,█▅▅▃▅▂▃▃▃▃▅▁▃
eval/precision,▂▆▃▅▄▁▇▇▆▆█▆▆
eval/recall,▂▁▄▄▃█▃▃▆▆▂▅▆
eval/runtime,▁▁▁▁▁█▁▁▁▁▁█▂
eval/samples_per_second,█████▁█████▁▇
eval/steps_per_second,█████▁█████▁▇
eval_accuracy,▁
eval_f1,▁


Run ffdb4zsk errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-5-24148b249e1b>", line 483, in sweep_train
    model, tokenizer = train_model(
  File "<ipython-input-5-24148b249e1b>", line 398, in train_model
    name=f"model_{model_id}_{run.id}",
UnboundLocalError: local variable 'run' referenced before assignment

wandb: ERROR Run ffdb4zsk errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-5-24148b249e1b>", line 483, in sweep_train
wandb: ERROR     model, tokenizer = train_model(
wandb: ERROR   File "<ipython-input-5-24148b249e1b>", line 398, in train_model
wandb: ERROR     name=f"model_{model_id}_{run.id}",
wandb: ERROR UnboundLocalError: local variable 'run' referenced 

In [14]:
# model, tokenizer = train_model(
#     train_dataset,   # Use the converted Dataset
#     val_dataset,     # Use the converted Dataset
#     2,               # model_id (choose 2 or 3)
#     model_hyperparameters,
#     model_args_dict,
#     WANDB_PROJECT_NAME
# )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/2558 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-13-5ec504b4366d>:366: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.464900,0.439751,0.802780,0.760837,0.883681,0.817671
200,0.414500,0.391337,0.811468,0.764749,0.900174,0.826954
300,0.380700,0.375238,0.825369,0.781955,0.902778,0.838034
400,0.359100,0.349964,0.833623,0.832900,0.835069,0.833984
500,0.348800,0.326207,0.845786,0.834593,0.862847,0.848485
600,0.315000,0.317671,0.849262,0.820207,0.894965,0.855957
700,0.306300,0.321117,0.854040,0.821260,0.905382,0.861272
800,0.322600,0.302583,0.856212,0.850555,0.864583,0.857512
900,0.283500,0.291234,0.862294,0.833733,0.905382,0.868082
1000,0.297300,0.304635,0.859253,0.839901,0.888021,0.863291


wandb: Adding directory to artifact (./type2)... Done. 2.6s


epoch,▁
eval/accuracy,▁▂▃▄▅▅▆▆▇▆▇██▆██
eval/f1,▁▂▃▃▅▅▆▆▇▆███▄██
eval/loss,█▆▅▄▃▃▃▂▁▂▃▁▁▂▁▁
eval/precision,▁▁▂▄▄▄▄▅▄▅▃▅▆█▆▅
eval/recall,▅▅▆▃▄▅▆▄▆▅█▆▅▁▅▆
eval/runtime,▁▃▅▆▇▆▇▇▇▆▇█▇█▇▆
eval/samples_per_second,█▅▃▃▂▂▂▂▂▂▂▁▂▁▂▃
eval/steps_per_second,█▅▃▃▂▂▂▂▂▂▂▁▂▁▂▃
eval_accuracy,▁
eval_f1,▁


In [ ]:
# model, tokenizer = train_model(
#     train_dataset,   # Use the converted Dataset
#     val_dataset,     # Use the converted Dataset
#     3,               # model_id (choose 2 or 3)
#     model_hyperparameters,
#     model_args_dict,
#     WANDB_PROJECT_NAME
# )